In [ ]:
import numpy as np
import math
import statsmodels.api as sm
import imblearn
from tqdm.notebook import tqdm
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite, community 
from numpy import nansum
from numpy import nanmean
from numpy.random import choice
import itertools
from networkx.algorithms import community
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
cb1 = pd.read_csv('cb1.csv')
cb2 = pd.read_csv('cb2.csv')
cb3 = pd.read_csv('cb3.csv')


In [ ]:
cb = cb1.append(cb2)
cb = cb.append(cb3)
cb.index = range(len(cb))
cb['date_my'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('m')
cb['date_y'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('y')

nodes_org = cb[['target_name', 'target_ID', 'target_country', 'target_continent','industry', 'target_founded', 'target_status']].drop_duplicates()
nodes_org['bipartite'] = 1
nodes_inv = cb[['investors_name', 'investor_id', 'investor_country', 'investor_continent', 'investor_type', 'investor_founded']].drop_duplicates()
nodes_org = nodes_org.rename(columns={"target_name": "node", "target_country": "country",  
                                      "target_continent": "continent", "industry":"industry", "target_status":"status",
                                     "target_ID":"index", "target_founded":"foundation"})
nodes_inv = nodes_inv.rename(columns={"investors_name": "node","investor_country": "country", 
                                      "investor_continent": "continent", "investor_type":"industry","investor_id":'index', "investor_founded":"foundation" })
nodes_inv['bipartite'] = 0
nodes = nodes_org.append(nodes_inv, ignore_index= True)
nodes['id'] = nodes.index
nodes_org = nodes[nodes['bipartite']==1]
nodes_inv = nodes[nodes['bipartite']==0]
numbers = cb.groupby(['target_name','stage'], as_index = False).count()[['target_name','stage','investors_name']].rename(columns={"investors_name": "investor_numbers"})
cb_new = pd.merge(cb, numbers,  how='left', left_on=['target_name','stage'], right_on = ['target_name','stage'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 1],  how='left', left_on=['target_name','target_ID'], right_on = ['node','index'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 0],  how='left', left_on=['investors_name','investor_id'], right_on = ['node','index'])


cb_new.date_y=cb_new.date_y.dt.year


In [119]:
B = nx.MultiGraph(name='org-bid')
# Add nodes with the node attribute "bipartite"
B.add_nodes_from(nodes[nodes.bipartite==0].id)
B.add_nodes_from(nodes[nodes.bipartite==1].id)
nx.set_node_attributes(B, pd.Series(nodes.node, index=nodes.id).to_dict(), 'name')
nx.set_node_attributes(B, pd.Series(nodes.country, index=nodes.id).to_dict(), 'country')
nx.set_node_attributes(B, pd.Series(nodes.continent, index=nodes.continent).to_dict(), 'continent')
nx.set_node_attributes(B, pd.Series(nodes.industry, index=nodes.id).to_dict(), 'industry')
nx.set_node_attributes(B, pd.Series(nodes.foundation, index=nodes.id).to_dict(), 'foundation')
nx.set_node_attributes(B, pd.Series(nodes.bipartite, index=nodes.id).to_dict(), 'bipartite')
nx.set_node_attributes(B, pd.Series(nodes.status, index=nodes.id).to_dict(), 'target_status')
# Add edges only between nodes of opposite node sets


In [120]:
for i in range(len(cb_new)):
    B.add_edge(cb_new.loc[i,'id_y'], cb_new.loc[i,'id_x'], date=(cb_new.loc[i,'date_y']), m_raised=cb_new.loc[i,'size_real'], 
               investors = cb_new.loc[i,'investor_numbers'], round_simp =cb_new.loc[i,'round_simp'])
brutti = [x for x,y in B.nodes(data=True) if (y['name'] in ['Undisclosed Investors', 'ciao']) or (y['name']=='Dead')   or (y['name']=='Public')  
          or  (y['name']=='Undisclosed Angel Investors') or (y['name']=='Undisclosed Venture Investors')]
B.remove_nodes_from(brutti)

In [ ]:
stati = cb_new.groupby('id_x')
stati2 = stati.tail(1).reset_index(drop=True)
stati3 = stati2[stati2["date"].notna()]

In [ ]:
stati3

In [35]:
stati2[stati2["target_status"] == "Merged"].drop_duplicates()[["target_name","round_simp","target_status", "date"]]

,target_name,round_simp,target_status,date
3692,Lankenau Institute for Medical Research,Merger,Merged,01/01/1985
4176,Global Pharmaceutical,IPO,Merged,12/19/1995
4240,BioClin International,Merger,Merged,08/19/1996
4256,Gensia Pharmaceuticals,Merger,Merged,11/01/1996
4320,OccuSystems,Merger,Merged,08/29/1997
...,...,...,...,...
49604,Hamilton County Systems of Care,Merger,Merged,07/15/2021
49605,Partnership For A Healthy Hamilton County,Merger,Merged,07/15/2021
49645,Fillmaster,Merger,Merged,07/16/2021
49646,FLAVORx,Merger,Merged,07/16/2021


In [37]:
fun_status = stati2[["round_simp", "target_status"]].groupby(['round_simp', "target_status"]).size().unstack(fill_value=0)[1:61]

In [45]:
fun_status.T.index.values

array(['Acquired', 'Alive / Active', 'Assets Purchased',
       'Dead / Inactive', 'IPO / Went public', 'Merged'], dtype=object)

In [94]:
fun_status.mode()['Dead / Inactive']
stage = list(fun_status.idxmax(axis=1).index)
stage.append("Acq - P2P")

stato = list(fun_status.idxmax(axis=1))
stato.append("Acquired")

In [93]:
set(stato)

{'Acquired',
 'Alive / Active',
 'Assets Purchased',
 'Dead / Inactive',
 'IPO / Went public',
 'Merged'}

In [69]:
def get_status(rounds):
    a = list()
    for i in rounds:
        indice = stage.index(i)
        a.append(stato[indice])
    return a
    
    


In [71]:
get_status(["Acquisition"])

['Acquired']

In [67]:
stage.index("Acquisition")

0

In [95]:
try: cb_new["stato"] = get_status(list(cb_new["round_simp"]))


In [96]:
cb_new["stato"]

0         IPO / Went public
1         IPO / Went public
2         IPO / Went public
3         IPO / Went public
4         IPO / Went public
                ...        
187799             Acquired
187800       Alive / Active
187801    IPO / Went public
187802             Acquired
187803             Acquired
Name: stato, Length: 187804, dtype: object

In [ ]:
stati.tail(1)

In [ ]:

remove=['Line of Credit',  'Loan']

cb_new=cb_new.loc[~cb_new.round_simp.isin(remove)]

In [ ]:
cb_new[14720:14724]

In [ ]:
#--------------------------------------

In [104]:
id_round = cb_new[['target_ID','round_simp']]
id_round = id_round.drop_duplicates()
id_round.index = range(len(id_round))
id_round['id_round'] = id_round.index
id_round

id_year = cb_new[['target_ID','date_y']]
id_year = id_year.drop_duplicates()
id_year.index = range(len(id_year))
id_year['id_year'] = id_year.index
id_year

id_stage = cb_new[['target_ID','stage']]
id_stage = id_stage.drop_duplicates()
id_stage.index = range(len(id_stage))
id_stage['id_stage'] = id_stage.index
id_stage

cb_new = pd.merge(cb_new, id_round,  how='left', left_on=['target_ID','round_simp'], right_on = ['target_ID','round_simp'])
cb_new = pd.merge(cb_new, id_stage,  how='left', left_on=['target_ID','stage'], right_on = ['target_ID','stage'])
cb_new = pd.merge(cb_new, id_year,  how='left', left_on=['target_ID','date_y'], right_on = ['target_ID','date_y'])

In [139]:
##bidbid in stesso stage
links = []
subcb = cb_new[['round_simp','date_y','id_y','size_real','id_stage','investor_numbers','target_name','id_x','stato','target_country']].drop_duplicates()
subcb = subcb[pd.notna(subcb['date_y'])]
subcb = subcb[pd.notna(subcb['size_real'])]
subcb = subcb[(subcb["round_simp"] != 'Line of Credit') | (subcb["round_simp"]!= 'Loan')]
for i in tqdm(subcb['id_stage'].unique()):
    subcb2 = subcb[subcb['id_stage']==i]
    for couple in itertools.combinations_with_replacement(np.unique(subcb2['id_y'].values),2):
        links.append((couple + ({'date': subcb2['date_y'].values[0],
                                'round': subcb2['round_simp'].values[0],
                                'money': subcb2['size_real'].values[0]/(subcb2['investor_numbers'].values[0]),
                                'investor_numbers': subcb2['investor_numbers'].values[0],
                                'status': subcb2["stato"].values[0],
                                 'id_target': subcb2["id_x"].values[0],
                                 'country_target': subcb2["target_country"].values[0],
                                'target_name': subcb2['target_name'].values[0]},)))

In [140]:
links

[(50685,
  50685,
  {'date': 1976,
   'round': 'Seed VC',
   'money': 491039.4265232975,
   'investor_numbers': 1,
   'status': 'Alive / Active',
   'id_target': 6693,
   'country_target': 'United States',
   'target_name': 'Genentech'}),
 (50989,
  50989,
  {'date': 1981,
   'round': 'Seed VC',
   'money': 2994535.5191256832,
   'investor_numbers': 1,
   'status': 'Alive / Active',
   'id_target': 6708,
   'country_target': 'United States',
   'target_name': 'Vestar Research'}),
 (54093,
  54093,
  {'date': 1981,
   'round': 'Seed VC',
   'money': 1026766.5952890792,
   'investor_numbers': 2,
   'status': 'Alive / Active',
   'id_target': 6710,
   'country_target': 'United States',
   'target_name': 'Diagnostic Technology'}),
 (54093,
  54094,
  {'date': 1981,
   'round': 'Seed VC',
   'money': 1026766.5952890792,
   'investor_numbers': 2,
   'status': 'Alive / Active',
   'id_target': 6710,
   'country_target': 'United States',
   'target_name': 'Diagnostic Technology'}),
 (54094,
  

In [141]:
bidbid_stage_self=nx.MultiGraph()
bidbid_stage_self.add_nodes_from(nodes[nodes.bipartite==0].id)
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.node, index=nodes_inv.id).to_dict(), 'name')
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.country, index=nodes_inv.id).to_dict(), 'country')
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.continent, index=nodes_inv.id).to_dict(), 'continent')
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.industry, index=nodes_inv.id).to_dict(), 'industry')
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.foundation, index=nodes_inv.id).to_dict(), 'foundation')
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.bipartite, index=nodes_inv.id).to_dict(), 'bipartite')
bidbid_stage_self.add_edges_from(links) 
remove = [node for node,degree in dict(bidbid_stage_self.degree()).items() if degree == 0]
bidbid_stage_self.remove_nodes_from(remove)
bidbid_stage_self.remove_nodes_from(brutti)
remove2 = [node for node,degree in dict(bidbid_stage_self.degree()).items() if degree == 0]
bidbid_stage_self.remove_nodes_from(remove2)
nx.readwrite.write_gpickle(bidbid_stage_self,'bidbid_stage_self_cb.gpickle')
nx.info(bidbid_stage_self)

'Name: \nType: MultiGraph\nNumber of nodes: 20413\nNumber of edges: 331456\nAverage degree:  32.4750'

In [151]:
bidbid_stage_self_single = nx.Graph()
for u,v,data in tqdm(bidbid_stage_self.edges(data=True)):
    d = data['date']
    m = data['money']
    r = data['round']
    t = data['target_name']
    
    if bidbid_stage_self_single.has_edge(u,v):
        bidbid_stage_self_single[u][v]['date'] = list(set([bidbid_stage_self[u][v][0]['date']] + [d]))
        bidbid_stage_self_single[u][v]['money'] += m
        bidbid_stage_self_single[u][v]['rounds'] = list(set([bidbid_stage_self[u][v][0]['round']] + [r]))
        bidbid_stage_self_single[u][v]['target_name'] = list(set([bidbid_stage_self[u][v][0]['target_name']] + [t]))
    else:
        bidbid_stage_self_single.add_edge(u, v, date=[d])
        bidbid_stage_self_single.add_edge(u, v, money = m)
        bidbid_stage_self_single.add_edge(u, v, rounds =[r])
        bidbid_stage_self_single.add_edge(u, v, target_name = [t])
        

In [152]:
nx.readwrite.write_gpickle(bidbid_stage_self_single,'bidbid_stage_self_single.gpickle')
nx.info(bidbid_stage_self_single)

'Name: \nType: Graph\nNumber of nodes: 20413\nNumber of edges: 164243\nAverage degree:  16.0920'

In [161]:
anni = range(1995,2022)
for periodo in anni:
    locals()["bidbid_stage_self_single_"+str(periodo-1)]=nx.Graph([(u,v,d) for u,v,d in bidbid_stage_self_single.edges(data=True) if d['date'][-1] < periodo])
    print(periodo-1)

1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [197]:
for periodo in anni:
    nx.readwrite.write_gpickle(locals()["bidbid_stage_self_single_"+str(periodo-1)], "bidbid_stage_self_single_"+str(periodo-1)+'.gpickle')
    print(nx.info(locals()["bidbid_stage_self_single_"+str(periodo-1)]))

Name: 
Type: Graph
Number of nodes: 382
Number of edges: 2601
Average degree:  13.6178
Name: 
Type: Graph
Number of nodes: 415
Number of edges: 2775
Average degree:  13.3735
Name: 
Type: Graph
Number of nodes: 456
Number of edges: 2939
Average degree:  12.8904
Name: 
Type: Graph
Number of nodes: 516
Number of edges: 3113
Average degree:  12.0659
Name: 
Type: Graph
Number of nodes: 788
Number of edges: 4299
Average degree:  10.9112
Name: 
Type: Graph
Number of nodes: 1070
Number of edges: 5797
Average degree:  10.8355
Name: 
Type: Graph
Number of nodes: 1798
Number of edges: 12817
Average degree:  14.2570
Name: 
Type: Graph
Number of nodes: 2266
Number of edges: 17345
Average degree:  15.3089
Name: 
Type: Graph
Number of nodes: 2646
Number of edges: 20923
Average degree:  15.8148
Name: 
Type: Graph
Number of nodes: 3015
Number of edges: 26594
Average degree:  17.6411
Name: 
Type: Graph
Number of nodes: 3436
Number of edges: 35291
Average degree:  20.5419
Name: 
Type: Graph
Number of nod

In [167]:
for u,v,d in bidbid_stage_self_single.edges(data=True):
    a = (d['date'][-1])

In [169]:
a<2021

False

In [194]:
for periodo in anni:
    print((locals()["bidbid_stage_self_single_"+str(periodo-1)]).number_of_edges())

2601
2775
2939
3113
4299
5797
12817
17345
20923
26594
35291
39897
44816
49089
52958
56634
60996
64978
68503
71842
75826
81543
89950
99178
112173
124587
144157


In [186]:
sorted(bidbid_stage_self_single_2020.degree, key=lambda x: x[1], reverse=True)

[(51253, 964),
 (50667, 742),
 (50481, 681),
 (55245, 572),
 (50919, 571),
 (50917, 569),
 (50770, 563),
 (50595, 544),
 (51276, 518),
 (50516, 499),
 (51077, 479),
 (53091, 471),
 (51084, 458),
 (51153, 452),
 (51305, 445),
 (56185, 432),
 (55127, 430),
 (51204, 429),
 (56017, 429),
 (55129, 423),
 (50766, 421),
 (50939, 420),
 (50531, 417),
 (54696, 415),
 (54697, 414),
 (50605, 412),
 (50685, 409),
 (50515, 407),
 (50859, 407),
 (50577, 405),
 (50519, 399),
 (51005, 397),
 (55133, 382),
 (51263, 368),
 (50874, 366),
 (52127, 365),
 (54057, 361),
 (51709, 357),
 (50683, 352),
 (50710, 351),
 (57908, 351),
 (51304, 348),
 (56477, 347),
 (56015, 345),
 (51211, 343),
 (50602, 337),
 (50832, 336),
 (51063, 329),
 (50554, 326),
 (51021, 323),
 (50505, 321),
 (50668, 321),
 (56967, 321),
 (50985, 317),
 (51175, 311),
 (54828, 310),
 (55131, 308),
 (54787, 298),
 (51512, 296),
 (50893, 296),
 (52419, 292),
 (50803, 291),
 (50581, 289),
 (52002, 286),
 (54704, 284),
 (57366, 283),
 (50768, 2

In [193]:
nodes[nodes['id'] == 50515]

,node,index,country,continent,industry,foundation,status,bipartite,id
50515,Abingworth,22678.0,United Kingdom,Europe,Venture Capital,NaN,NaN,0,50515
